In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from collections import defaultdict, Counter
import re

In [2]:
switch_router = {'s1':'r1 r2', 's2':'r2 r3', 's3':'r3 r4', 's4':'r4 r5', 's5':'r5 r6', 's6':'r6 r1', 's7':'r7 r8',
                's8':'r8 r9', 's9':'r9 r10', 's10':'r10 r7', 's11':'r1 r8', 's12':'r3 r8', 's13':'r3 r9',
                's14':'r4 r10', 's15':'r6 r10', 's16':'r6 r7'}
FILES_DIR = './'

In [3]:
dir_content = os.listdir(FILES_DIR)
predictions = sorted([f for f in dir_content if 'loss_' in f])
mininet_dumps = sorted([f for f in dir_content if 'mininet_dump_' in f])

In [31]:
def get_path(line):
    path = line.split(':')[1]
    path = re.sub('[\'\[\],]','', path)#.split()
    return path

# string with the mininet dump containing the loss of each interface on the switches
def get_link_w_loss(mn_loss):
    step = 7
    idx = np.arange(0,len(mn_loss.split()), step)
    losses = [mn_loss.split()[i:i+step] for i in idx]
    for l in losses:
        if l[2]!= '0%':
            print(l)
    print()
    
def print_counter(dic):
    for k, i in dic.items():
        print(k, Counter(i))
    print()

In [32]:
for pr, mn in zip(predictions, mininet_dumps):
    ospf = {}
    pred = defaultdict(list)
    link_loss='********'
    with open(os.path.join(FILES_DIR, pr), 'r') as pr_f, open(os.path.join(FILES_DIR, mn), 'r') as mn_f:
        for i,line in enumerate(pr_f):
            line = line.strip()
            if 'ITERATION' in line:
                # starting new iteration
                if ospf and pred:
                    #print(ospf)
                    get_link_w_loss(link_loss)
                    #print_counter(pred)
                ospf = {}
                pred = defaultdict(list)
                # getting the link loss
                link_loss = mn_f.readline().strip()
                while '*' in link_loss:
                    link_loss = mn_f.readline().strip()
                
            # matching the target of the prediction
            elif re.match('r(\d+)_(\d+)_(\d+)_(\d+)_(\d+)', line):
                target = line
            
            elif 'prediction' in line:
                pred[target].append(get_path(line))
            
            elif 'ospf' in line:
                if target not in ospf:
                    ospf[target]=get_path(line)
            else:
                pass
        #print_counter(pred)
        print('*'*30)
        break


['s4', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s7', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s11', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s12', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s15', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']

['s2', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s3', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s6', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s9', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s11', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']

['s1', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s2', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s5', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s7', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%', 'loss)']
['s8', '(300.00Mbit', '5%', 'loss)', '(300.00Mbit', '5%'